# Prediction Delivery Time

# EDA Dataset

## Explore

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime, timezone


In [6]:
df_ebay = pd.read_csv('../data/raw/ebayShort.csv', nrows=100000 ,index_col=0)

As the data is too big, we first working in the shortcut of the total data with 1 million rows.

In [7]:
# Look at the first 5 row

df_ebay.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [8]:
#Look at 5 random row
df_ebay.sample(5)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
44578,B2C,349514,0.0,2019-08-27 00:49:00.000-05:00,0,0.0,3,5,77433,22958,11,44.99,1,2019-08-25 12:06:52.000-05:00,2019-08-30,4,1,PACKAGE_THICK_ENVELOPE,44579
12674,C2C,355909,1.0,2019-07-12 10:19:00.000-04:00,0,0.0,3,5,06902,76859,12,24.95,1,2019-07-11 11:17:54.000-04:00,2019-07-16,3,1,PACKAGE_THICK_ENVELOPE,12675
15476,B2C,831,1.0,2018-03-19 22:37:00.000-07:00,0,0.0,3,5,91786,11731,5,8.99,1,2018-03-16 14:49:04.000-07:00,2018-03-21,0,1,LETTER,15477
46190,B2C,476,1.0,2018-04-29 00:16:00.000-07:00,0,0.0,3,5,518000,75052,8,1.28,1,2018-04-27 12:50:25.000-07:00,2018-05-03,0,1,PACKAGE_THICK_ENVELOPE,46191
15137,B2C,64668,3.0,2018-01-18 11:44:00.000-08:00,2,6.0,2,9,94404,38138,8,26.00,1,2018-01-17 05:40:34.000-08:00,2018-01-20,0,1,LETTER,15138


In [9]:
#check the shape of the dataset
df_ebay.shape

(100000, 19)

In [10]:
print(f'There are {df_ebay.shape[0]} columns and {df_ebay.shape[1]} rows')

There are 100000 columns and 19 rows


In [11]:
# Get a quick overview of dataset variables
df_ebay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   b2c_c2c                    100000 non-null  object 
 1   seller_id                  100000 non-null  int64  
 2   declared_handling_days     95042 non-null   float64
 3   acceptance_scan_timestamp  100000 non-null  object 
 4   shipment_method_id         100000 non-null  int64  
 5   shipping_fee               100000 non-null  float64
 6   carrier_min_estimate       100000 non-null  int64  
 7   carrier_max_estimate       100000 non-null  int64  
 8   item_zip                   100000 non-null  object 
 9   buyer_zip                  100000 non-null  object 
 10  category_id                100000 non-null  int64  
 11  item_price                 100000 non-null  float64
 12  quantity                   100000 non-null  int64  
 13  payment_datetime           100

In [12]:
# top 5 rows showing only 'object' columns
df_ebay.select_dtypes('object').head()

,b2c_c2c,acceptance_scan_timestamp,item_zip,buyer_zip,payment_datetime,delivery_date,package_size
0,B2C,2019-03-26 15:11:00.000-07:00,97219,49040,2019-03-24 03:56:49.000-07:00,2019-03-29,LETTER
1,C2C,2018-06-02 12:53:00.000-07:00,11415-3528,62521,2018-06-01 13:43:54.000-07:00,2018-06-05,PACKAGE_THICK_ENVELOPE
2,B2C,2019-01-07 16:22:00.000-05:00,27292,53010,2019-01-06 00:02:00.000-05:00,2019-01-10,PACKAGE_THICK_ENVELOPE
3,B2C,2018-12-17 16:56:00.000-08:00,90703,80022,2018-12-16 10:28:28.000-08:00,2018-12-21,PACKAGE_THICK_ENVELOPE
4,B2C,2018-07-27 16:48:00.000-07:00,90703,55070,2018-07-26 18:20:02.000-07:00,2018-07-30,PACKAGE_THICK_ENVELOPE


Let see the percent of the kind of customer B2C(Business to Consumer) and C2C(Consumer to Consumer)

In [13]:
df_ebay['b2c_c2c'].value_counts()

B2C    69720
C2C    30280
Name: b2c_c2c, dtype: int64

In [16]:
fig= px.pie(df_ebay, values= df_ebay['b2c_c2c'].value_counts(), names=df_ebay['b2c_c2c'].unique())

fig.show()

Seem like most of the seller is the business which take about 69.7% of the total amount of sale in the online selling platform. Consumer to consumer takes about 30.3% of the total amount, which is not a small amount, that we can see that there are more and more people start using the online platform to sell their product.

Let see how the seller declared the amount of time they need to proceesing order to make it ready to ship for customer base on there weight of the package>

In [17]:
df_ebay.groupby('declared_handling_days')['weight'].count().sort_values()

declared_handling_days
20.0       64
30.0       68
15.0       99
10.0      411
4.0       760
5.0      1460
0.0      6907
3.0     16582
2.0     17850
1.0     50841
Name: weight, dtype: int64

In [11]:
fig= px.scatter(df_ebay, x= 'declared_handling_days', y= df_ebay['shipment_method_id'])
fig.show()

## Data Cleaning

First let do some common checking in the dataset.
 -  Null value
 -  Duplicate row

In [18]:
#Checking null
df_ebay.isna().sum()

b2c_c2c                         0
seller_id                       0
declared_handling_days       4958
acceptance_scan_timestamp       0
shipment_method_id              0
shipping_fee                    0
carrier_min_estimate            0
carrier_max_estimate            0
item_zip                        0
buyer_zip                       0
category_id                     0
item_price                      0
quantity                        0
payment_datetime                0
delivery_date                   0
weight                          0
weight_units                    0
package_size                    0
record_number                   0
dtype: int64

Drop all null or missing value:

In [26]:
df_ebay=df_ebay.dropna()

In [27]:
df_ebay.isna().sum()

b2c_c2c                      0
seller_id                    0
declared_handling_days       0
acceptance_scan_timestamp    0
shipment_method_id           0
shipping_fee                 0
carrier_min_estimate         0
carrier_max_estimate         0
item_zip                     0
buyer_zip                    0
category_id                  0
item_price                   0
quantity                     0
payment_datetime             0
delivery_date                0
weight                       0
weight_units                 0
package_size                 0
record_number                0
dtype: int64

There is 47180 missing value in total 1 million rows. We could delete it but I decide to have deepper look at it later


In [28]:
# checking duplicate rows
df_ebay.duplicated().any()

False

There are no duplicate row in the database

In [29]:
#recall shape column
df_ebay.shape[1]

19

In [30]:
#Dataset
df_ebay.head(5)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


Let check out the distributions, we are going to check over all the column in the dataset

In [31]:
#checking the b2c_c2c column
df_ebay['b2c_c2c'].value_counts()

B2C    66944
C2C    28098
Name: b2c_c2c, dtype: int64

Convert `b2c_c2c` column to binary

In [32]:
df_ebay['b2c_c2c']= np.where(df_ebay['b2c_c2c']=='B2C', 1, 0)

Test it

In [33]:
df_ebay.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,1,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,0,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,1,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,1,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,1,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


Check the seller_id Colluum


In [34]:
df_ebay['seller_id'].value_counts()

0         1059
11         191
1          182
4          140
40          99
          ... 
91582        1
301906       1
113882       1
10213        1
311534       1
Name: seller_id, Length: 54912, dtype: int64

In [35]:
#Check null
df_ebay['seller_id'].isna().any()

False

There is 2 `weight_units` is 1 and 2. Let say 1 - 's' and 2 - 'kg'. We are going to convert all the weight units to lb.

In [36]:
#check the record has weigtht unit =2 to able to compare
df_ebay[df_ebay['weight_units']==2].head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number


In [37]:
df_ebay['weight'] = np.where(df_ebay['weight_units']==1, df_ebay['weight'], df_ebay['weight']*2.20462 )

check the record with `weight_units`=2 to see if the `weight` column is change

In [38]:
df_ebay[df_ebay['weight_units']==2]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number


All the `weight` with the `weight_units` = 2 are convert to the **lb** unit. So now we do not need to keep the weight column since all the weight is in the **lb**. Therefore, we are going to drop the `weight column`.

In [39]:
df_ebay.drop(columns='weight', axis=1, inplace=True)


In [40]:
#check it
df_ebay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95042 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   b2c_c2c                    95042 non-null  int64  
 1   seller_id                  95042 non-null  int64  
 2   declared_handling_days     95042 non-null  float64
 3   acceptance_scan_timestamp  95042 non-null  object 
 4   shipment_method_id         95042 non-null  int64  
 5   shipping_fee               95042 non-null  float64
 6   carrier_min_estimate       95042 non-null  int64  
 7   carrier_max_estimate       95042 non-null  int64  
 8   item_zip                   95042 non-null  object 
 9   buyer_zip                  95042 non-null  object 
 10  category_id                95042 non-null  int64  
 11  item_price                 95042 non-null  float64
 12  quantity                   95042 non-null  int64  
 13  payment_datetime           95042 non-null  obj

The `weigth` column has been dropped. Next we are going to calculate the distance between the buyer and the seller buy using there zip code in the `item_zip` and `buyer_zip` columns.

Check the `package_size` column

In [41]:
#How many type of package 
df_ebay['package_size'].value_counts()

PACKAGE_THICK_ENVELOPE    80022
NONE                       6866
LETTER                     5604
LARGE_ENVELOPE             1445
LARGE_PACKAGE              1105
Name: package_size, dtype: int64

There are 5 type of package. so we can use the ordinal encoding to tranfer this column to numeric instead of category column. `'NONE': 0, 'LETTER': 1, 'LARGE_ENVELOPE': 2, 'LARGE_PACKAGE': 3, 'PACKAGE_THICK_ENVELOPE': 4`

In [42]:
oe_package= OrdinalEncoder(categories=[['NONE', 'LETTER', 'LARGE_ENVELOPE',
                                         'LARGE_PACKAGE', 'PACKAGE_THICK_ENVELOPE']])
df_ebay['package_size']=oe_package.fit_transform(pd.DataFrame(df_ebay['package_size'])) 

In [43]:
df_ebay['package_size'].value_counts()

4.0    80022
0.0     6866
1.0     5604
2.0     1445
3.0     1105
Name: package_size, dtype: int64

In [44]:
#check any null value in these columns.
print(df_ebay['item_zip'].isna().any())
print(df_ebay['buyer_zip'].isna().any())

False
False


In [45]:
item_zip= df_ebay['item_zip']
buyer_zip= df_ebay['buyer_zip']

In [46]:
import mpu
from uszipcode import SearchEngine

def get_distance(item_zip, buyer_zip):
    """
    We are going to use the package mpu and uszipcode to imporve the speed of calculating the distance between buyer and seller by using zip code. 
    These 2 packages make the speed of calculation much faster compare with using the geopy package
    """
    if item_zip is not None and buyer_zip is not None:
        search = SearchEngine()
        item_location = search.by_zipcode(item_zip[0:5])
        buyer_location =search.by_zipcode(buyer_zip[0:5])
        if item_location is None or buyer_location is None:
            return None
        else:
            lat1 =item_location.lat
            long1 =item_location.lng
            lat2 =buyer_location.lat
            long2 =buyer_location.lng
            if lat1 is None or lat2 is None or long1 is None or long2 is None:
                return None
            return mpu.haversine_distance((lat1,long1),(lat2,long2)) 
    else:
        return None


def add_zip_distance_column(item_zip, buyer_zip):
    item_zip_str = item_zip.apply(lambda x: str(x))
    buyer_zip_str = buyer_zip.apply(lambda x: str(x))

    zips = pd.concat([item_zip_str, buyer_zip_str], axis=1)

    zips['distance'] = zips.apply(lambda x: get_distance(x.item_zip, x.buyer_zip), axis=1)
    
    return zips['distance']

In [47]:
distance = add_zip_distance_column(item_zip, buyer_zip)

Add the value of the distance in to the data frame

In [54]:
df_ebay['distance']= distance
df_ebay.head(10)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight_units,package_size,record_number,distance
0,1,25454,3.0,2019-03-26 15:11:00,0,0.00,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,1,1.0,1,3001.839179
1,0,6727381,2.0,2018-06-02 12:53:00,0,3.00,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,1,4.0,2,1282.908017
2,1,18507,1.0,2019-01-07 16:22:00,0,4.50,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,1,4.0,3,1104.444168
3,1,4677,1.0,2018-12-17 16:56:00,0,0.00,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,1,4.0,4,1353.390003
4,1,4677,1.0,2018-07-27 16:48:00,0,0.00,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,1,4.0,5,2456.330752
5,1,10514,1.0,2019-04-19 19:42:00,0,0.00,3,5,43215,77063,3,10.39,1,2019-04-18 14:11:09.000-04:00,2019-04-22,1,4.0,6,1608.340906
6,1,104,1.0,2019-02-08 17:35:00,0,0.00,3,5,91304,60565,11,5.70,1,2019-02-08 09:33:13.000-08:00,2019-02-11,1,4.0,7,2781.527939
7,1,340356,1.0,2018-04-23 17:31:00,0,2.95,3,5,49735,29379,1,6.00,1,2018-04-22 18:32:04.000-04:00,2018-04-25,1,4.0,8,1174.270246
8,1,113915,5.0,2019-10-12 09:22:00,3,0.00,2,8,43606,32958,18,5.55,1,2019-10-11 04:54:25.000-04:00,2019-10-15,1,0.0,9,1568.867800
9,1,130301,1.0,2019-08-09 11:24:00,1,0.00,2,5,35117,84776,13,59.98,1,2019-08-08 12:47:14.000-05:00,2019-08-12,1,4.0,10,2311.015450


Working with `acceptance_scan_timestamp` and `payment_datetime`. convert all in to buyer timezone

In [55]:
#Recall these 2 columns
df_ebay[['acceptance_scan_timestamp', 'payment_datetime']]

,acceptance_scan_timestamp,payment_datetime
0,2019-03-26 15:11:00,2019-03-24 03:56:49.000-07:00
1,2018-06-02 12:53:00,2018-06-01 13:43:54.000-07:00
2,2019-01-07 16:22:00,2019-01-06 00:02:00.000-05:00
3,2018-12-17 16:56:00,2018-12-16 10:28:28.000-08:00
4,2018-07-27 16:48:00,2018-07-26 18:20:02.000-07:00
...,...,...
99995,2018-07-23 17:38:00,2018-07-22 12:35:12.000-07:00
99996,2018-04-17 10:16:00,2018-04-16 08:55:56.000-05:00
99997,2018-04-27 11:41:00,2018-04-26 11:52:36.000-04:00
99998,2018-08-20 21:05:00,2018-08-20 10:31:39.000-04:00


In [64]:
accept_date= pd.to_datetime(df_ebay['acceptance_scan_timestamp'])#.replace(tzinfo= timezone.utc).astimezone(tz=None)
pay_date= pd.to_datetime(df_ebay['payment_datetime'])#.replace(tzinfo= timezone.utc).astimezone(tz=None)
delivery_date= pd.to_datetime(df_ebay["delivery_date"])

In [108]:
#Date convert function
def convert_date_format(date_list):
    output_format = "%Y-%m-%d %H:%M:%S"
    dates=[]
    # input_date= datetime.strftime(date[0], input_format)
    for day in date_list:    
        dates.append(pd.to_datetime(datetime.strftime(day, output_format)))
    return dates

In [109]:
#convert the `acceptance_scan_timestamp` column
df_ebay['acceptance_scan_timestamp']= convert_date_format(accept_date)

#convert `payment_datetime` column
df_ebay['payment_datetime']= convert_date_format(pay_date)

#convert  `delivery_date` column

df_ebay['delivery_date']= convert_date_format(delivery_date)

In [70]:
df_ebay['acceptance_scan_timestamp']

0        2019-03-26 15:11:00
1        2018-06-02 12:53:00
2        2019-01-07 16:22:00
3        2018-12-17 16:56:00
4        2018-07-27 16:48:00
                ...         
99995    2018-07-23 17:38:00
99996    2018-04-17 10:16:00
99997    2018-04-27 11:41:00
99998    2018-08-20 21:05:00
99999    2018-07-23 11:58:00
Name: acceptance_scan_timestamp, Length: 95042, dtype: object

In [71]:
df_ebay['payment_datetime']

0        2019-03-24 03:56:49
1        2018-06-01 13:43:54
2        2019-01-06 00:02:00
3        2018-12-16 10:28:28
4        2018-07-26 18:20:02
                ...         
99995    2018-07-22 12:35:12
99996    2018-04-16 08:55:56
99997    2018-04-26 11:52:36
99998    2018-08-20 10:31:39
99999    2018-07-21 18:10:24
Name: payment_datetime, Length: 95042, dtype: object

In [110]:
type(df_ebay['delivery_date'][1])

pandas._libs.tslibs.timestamps.Timestamp

Take a look at th data  after change

In [125]:
df_ebay.head(10)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight_units,package_size,record_number,distance
0,1,25454,3.0,2019-03-26 15:11:00,0,0.00,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49,2019-03-29,1,1.0,1,3001.839179
1,0,6727381,2.0,2018-06-02 12:53:00,0,3.00,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54,2018-06-05,1,4.0,2,1282.908017
2,1,18507,1.0,2019-01-07 16:22:00,0,4.50,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00,2019-01-10,1,4.0,3,1104.444168
3,1,4677,1.0,2018-12-17 16:56:00,0,0.00,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28,2018-12-21,1,4.0,4,1353.390003
4,1,4677,1.0,2018-07-27 16:48:00,0,0.00,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02,2018-07-30,1,4.0,5,2456.330752
5,1,10514,1.0,2019-04-19 19:42:00,0,0.00,3,5,43215,77063,3,10.39,1,2019-04-18 14:11:09,2019-04-22,1,4.0,6,1608.340906
6,1,104,1.0,2019-02-08 17:35:00,0,0.00,3,5,91304,60565,11,5.70,1,2019-02-08 09:33:13,2019-02-11,1,4.0,7,2781.527939
7,1,340356,1.0,2018-04-23 17:31:00,0,2.95,3,5,49735,29379,1,6.00,1,2018-04-22 18:32:04,2018-04-25,1,4.0,8,1174.270246
8,1,113915,5.0,2019-10-12 09:22:00,3,0.00,2,8,43606,32958,18,5.55,1,2019-10-11 04:54:25,2019-10-15,1,0.0,9,1568.867800
9,1,130301,1.0,2019-08-09 11:24:00,1,0.00,2,5,35117,84776,13,59.98,1,2019-08-08 12:47:14,2019-08-12,1,4.0,10,2311.015450


Now we see that most of the column is in the good shape. But the model cannot run with the datetime type of variable. So we are going to create some 2 new column base on the datetime column. 
-   **handling_date** = `acceptance_scan_timestamp` - `payment_dateime`
-   **shipping_date** = `delivery_date` - `acceptance_scan_timestamp`

In [121]:
#calulate date function
def date_calculate(day1, day2):
    number_of_day=[]
    date_list= day2- day1
    for day in date_list:
        date= day.days
        if day.seconds/3600 >=12:
            date+=1
        number_of_day.append(date)
    return number_of_day
        


In [129]:
#calulate handling_date
handling_date= date_calculate(df_ebay['payment_datetime'], df_ebay['acceptance_scan_timestamp'])

#calculate shipping_date
shipping_date= date_calculate(df_ebay['acceptance_scan_timestamp'], df_ebay['delivery_date'])


Create `handling_date` and `shipping_date` as new columns in data frame and add the value for it

In [130]:
df_ebay['handling_date']= handling_date
df_ebay['shipping_date']= shipping_date

In [132]:
#check data frame
df_ebay.sample(10)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,...,item_price,quantity,payment_datetime,delivery_date,weight_units,package_size,record_number,distance,handling_date,shipping_date
27755,1,5998,3.0,2018-11-02 20:38:00,0,0.00,3,5,53188,20036,...,30.00,1,2018-10-31 20:59:48,2018-11-05,1,4.0,27756,1048.162935,2,2
72132,1,1810,1.0,2018-03-14 17:07:00,0,0.00,3,5,75002,02368,...,5.69,1,2018-03-14 09:40:23,2018-03-17,1,4.0,72133,2457.903018,0,2
25780,1,3359,3.0,2019-02-22 21:34:00,0,0.00,3,5,90001,40701,...,9.79,1,2019-02-21 15:09:06,2019-02-25,1,0.0,25781,3093.414249,1,2
81453,0,665425,2.0,2019-04-03 04:54:00,6,0.00,2,5,36105,40069,...,29.92,1,2019-04-02 13:03:56,2019-04-06,1,0.0,81454,615.991026,1,3
24063,0,63077,1.0,2019-09-04 22:01:00,3,3.75,2,8,98671,67211,...,7.00,1,2019-09-04 03:23:58,2019-09-10,1,4.0,24064,2238.590235,1,5
21349,1,33705,3.0,2019-02-02 15:51:00,0,0.00,3,5,95037,12528,...,19.99,1,2019-01-30 15:01:55,2019-02-05,1,4.0,21350,4073.913341,3,2
87127,0,42666,1.0,2018-02-08 14:59:00,0,4.99,3,5,21632,65775,...,12.00,1,2018-02-08 09:50:50,2018-02-12,1,4.0,87128,1429.908469,0,3
13776,1,187,1.0,2019-10-07 14:56:00,0,0.00,3,5,32561,32092,...,5.53,1,2019-10-05 15:55:02,2019-10-09,1,4.0,13777,546.160963,2,1
63726,1,66246,1.0,2018-07-23 15:38:00,0,0.00,3,5,92503,42040,...,12.99,1,2018-07-21 16:31:39,2018-07-25,1,4.0,63727,2632.370208,2,1
21653,0,1659502,1.0,2019-03-18 20:02:00,5,8.00,2,5,32312,32773,...,10.00,1,2019-03-15 21:43:57,2019-03-20,1,4.0,21654,350.905807,3,1


In [137]:
df_ebay.shape

(95042, 21)

In [135]:
#check null again
df_ebay.isna().sum()

b2c_c2c                         0
seller_id                       0
declared_handling_days          0
acceptance_scan_timestamp       0
shipment_method_id              0
shipping_fee                    0
carrier_min_estimate            0
carrier_max_estimate            0
item_zip                        0
buyer_zip                       0
category_id                     0
item_price                      0
quantity                        0
payment_datetime                0
delivery_date                   0
weight_units                    0
package_size                    0
record_number                   0
distance                     1257
handling_date                   0
shipping_date                   0
dtype: int64

We can see there are some value in the `distance` column is count as NA here. It could be because the Zipcode of the buyer or seller is not in the right format. We are going to drop all of row with NA in distance here since it is only 1257 rows over 95042 rows in total(around 1.3% the amount of data).

In [140]:
df_ebay= df_ebay.dropna()
df_ebay.isna().any().any()

False

Now we export the data to the new CSV then later on we only work with clean data

In [141]:
df_ebay.to_csv('../data/cleaned/Ebay_cleaned.csv')